# 
This jupyter notebook used for analyzing the single kluverboard 

In [ ]:
import deeplabcut
import os
import time
import csv
from tkinter import filedialog
import glob

### Set up various tags

+ project exist, train data set exist
+ frames extracted, frames labeled
+ need training, evaluation, and analyze functions or not
+ need create label function or note

In [ ]:
# tag for create a new project (True) or not (False)
tag_createprj = False

# tags for extract frames (True) or not (False)
tag_extractframes = False

# tag for label frames (True) or not (False)
tag_labelframes = True

# tag for needing train (True) or not (False)
tag_train = True 

# tag for evaluating trained model(True) or not (False)
tag_evaluation = True


# tag for whether analyze new videos (True) or not (False)
tag_analyze = False

# tag for creating labels for analyzed videos (True) or not (False)
tag_createlabel = True

### Extract the absoluate path of config file config.yaml

Create new project if tag_prjexist = False, otherwise input the project directory

In [ ]:
task = 'PinkySKB' 
experimenter = 'YLL'

# working directory where the project directory will be created
working_dir = 'dlcProjects'
if tag_createprj:
    """ 
    create a new project 
    """ 
    
    
    # initial directory of dialog for video input
    root2_dir = '/run/user/1000/gvfs/ftp:host=nmrc_dserver1.local/root2'
    diaginitdir = os.path.join(root2_dir, 'Animals2', 'Pinky', 'Recording', 'Raw', 'rawMA')
    # dialog for input the videos to be used
    videos = filedialog.askopenfilenames(initialdir = diaginitdir, title = 'Please select videos to be used',
                                       filetypes = (("video files","*.avi"),("all files","*.*")))
    
    # deeolapcut create_new_project func
    path_config_file = deeplabcut.create_new_project(task, experimenter, videos, 
                                                     working_directory = working_dir, copy_videos = True)
    
    # extract the project path and the config file name
    path_prj, config_file_name = os.path.split(path_config_file)
    
else:
    """ 
    project already exist 
    """  
    # dialog for input the exist project folder
    path_prj = filedialog.askdirectory(initialdir = os.getcwd(), title = 'Please select the exist project folder')
    
    # dlc config file config.yaml
    path_config_file = os.path.join(path_prj, 'config.yaml')
    
    if not os.path.exists(path_config_file):
        print('No config.yaml file in ' + path_prj)

### Edit the config.yaml to config the project
+ bodyparts: Hand, Startpad, Mouth, Food


### Extract and label the frames
+ Random frames are extracted from all the videos in the project configuration file in order to create a training dataset.

+ 'automatic' and 'manual' extract frames are used once to extract the frames now. 

In [ ]:
""" extract the frames"""
if tag_extractframes:
    deeplabcut.extract_frames(path_config_file, 'automatic', 'uniform',crop=False)
    
    # maunal extract frames
    deeplabcut.extract_frames(path_config_file, 'manual', 'uniform',crop=False)
    

""" label the frames"""
if tag_labelframes:
    deeplabcut.label_frames(path_config_file) # label frames

# train the dlc model



In [ ]:
if tag_train:
    
    # created training dataset if training dateset doesn't exist
    if not glob.glob(os.path.join(path_prj, 'training-datasets', 'iteration-*')):
        deeplabcut.check_labels(path_config_file)
        deeplabcut.create_training_dataset(path_config_file, num_shuffles=1)
    
    # train the dlc network
    train_start = time.time()
    deeplabcut.train_network(path_config_file, shuffle=1)
    train_end = time.time()
    train_time = train_end - train_start
    
    print('training time is %f' %train_time)

# Evaluation models

In [ ]:
if tag_evaluation:
    deeplabcut.evaluate_network(path_config_file, plotting = True)

# Analyze new videos 

In [ ]:
# analyze new video
if tag_analyze:
    
    # dialog for input the videos to be analyzed
    videos_tobeanalyzed = filedialog.askopenfilenames(title = 'Please select videos to be analyzed',
                                       filetypes = (("video files","*.avi"),("all files","*.*")))
    
    deeplabcut.analyze_videos(path_config_file, videos_tobeanalyzed,save_as_csv = True, videotype = '.avi')

# Create labelled video

In [ ]:
# create labeled video
if tag_createlabel:
    # dialog for input the videos to be labelled
    video_tobelabelled = filedialog.askopenfilenames(title = 'Please select videos to be labelled',
                                       filetypes = (("video files","*.avi"),("all files","*.*")))
    deeplabcut.create_labeled_video(path_config_file,video_tobelabelled)